# Running Complete Spark Statements


reading "2019.csv"


In [ ]:
df = spark.read.csv("Files/2019.csv")

displaying 2019.csv


In [ ]:
display(df)

defining schema for 2019.csv

In [ ]:
from pyspark.sql.types import *

orderSchema = StructType([
    StructField("SalesOrderNumber", StringType()),
    StructField("SalesOrderLineNumber", IntegerType()),
    StructField("OrderDate", DateType()),
    StructField("CustomerName", StringType()),
    StructField("Email", StringType()),
    StructField("Item", StringType()),
    StructField("Quantity", IntegerType()),
    StructField("UnitPrice", FloatType()),
    StructField("Tax", FloatType())
])

df = spark.read.format("csv").schema(orderSchema).load("Files/2019.csv")

display(df)

defining schema and then reading the files altogether

In [ ]:
from pyspark.sql.types import *

orderSchema = StructType([
    StructField("SalesOrderNumber", StringType()),
    StructField("SalesOrderLineNumber", IntegerType()),
    StructField("OrderDate", DateType()),
    StructField("CustomerName", StringType()),
    StructField("Email", StringType()),
    StructField("Item", StringType()),
    StructField("Quantity", IntegerType()),
    StructField("UnitPrice", FloatType()),
    StructField("Tax", FloatType())
    ])

df = spark.read.format("csv").schema(orderSchema).load("Files/*.csv")

display(df)

playing with dataframe operations

In [ ]:
customers = df['CustomerName', 'Email']

print(customers.count())
print(customers.distinct().count())

display(customers.distinct())

In [ ]:
customers = df.select("CustomerName", "Email").where(df['Item']=='Road-250 Red, 52')
print(customers.count())
print(customers.distinct().count())

display(customers.distinct())

In [ ]:
productSales = df.select("Item", "Quantity").groupBy("Item").sum()

display(productSales)

In [ ]:
from pyspark.sql.functions import split

# Select only "CustomerName" and "Email" columns (assuming they exist in your original DataFrame 'df')
customers_with_names = df.select("CustomerName", "Email")

# Split the "CustomerName" column using a space delimiter
customers_with_names = customers_with_names.withColumn("firstName", split(customers_with_names["CustomerName"], " ").getItem(0))
customers_with_names = customers_with_names.withColumn("lastName", split(customers_with_names["CustomerName"], " ").getItem(1))

# Display the resulting DataFrame
display(customers_with_names)

In [ ]:
from pyspark.sql.functions import *

yearlySales = df.select(year(col("OrderDate")).alias("Year")).groupBy("Year").count().orderBy("Year")

display(yearlySales)

transforming the data to be saved in parquet file format


In [ ]:
from pyspark.sql.functions import *

# Create Year and Month columns
transformed_df = df.withColumn("Year", year(col("OrderDate"))).withColumn("Month", month(col("OrderDate")))

# Create the new FirstName and LastName fields
transformed_df = transformed_df.withColumn("FirstName", split(col("CustomerName"), " ").getItem(0)).withColumn("LastName", split(col("CustomerName"), " ").getItem(1))

# Filter and reorder columns
transformed_df = transformed_df["SalesOrderNumber", "SalesOrderLineNumber", "OrderDate", "Year", "Month", "FirstName", "LastName", "Email", "Item", "Quantity", "UnitPrice", "Tax"]

# Display the first five orders
display(transformed_df.limit(5))

saving the transformed data in parquet formatted file


In [ ]:
transformed_df.write.mode("overwrite").parquet('Files/transformed_data/orders')

print ("Transformed data saved!")

loading the parquet formatted file in a dataframe

In [ ]:
orders_df = spark.read.format("parquet").load("Files/transformed_data/orders")
display(orders_df)

saving as a delta table

In [ ]:
orders_df.write.format("delta").saveAsTable("salesOrders")

using spark.sql API support to read the delta table


In [ ]:
sql_df = spark.sql("SELECT * FROM salesorders")
display(sql_df)

# Running complete SQL statements


In [ ]:
%%sql
SELECT YEAR(OrderDate) AS OrderYear,
       SUM((UnitPrice * Quantity) + Tax) AS GrossRevenue
FROM salesorders
GROUP BY YEAR(OrderDate)
ORDER BY OrderYear;